In [112]:
%load_ext autoreload
%autoreload 2

from denoised_dataset import TimeVocab, DenoisingDataset
import torch

vocab = TimeVocab(torch.zeros(1000, 250, 100), torch.tensor([1, 2, 3]), torch.tensor([1, 2, 3]), 64, 120)

print(len(vocab))

dataset = DenoisingDataset(
    vocab,
    127,
    mask=0.3,
    mask_random=0.3,
    insert=0,
    rotate=0,
    permute_sentences=0,
    replace_length=-1,
    mask_length='span-poisson',
    poisson_lambda=3
)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
1004


In [81]:
print(dataset.dataset[0])

tensor([  1,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,  14,  15,  16,
         17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,  28,  29,  30,
         31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,  42,  43,  44,
         45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,  56,  57,  58,
         59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,  70,  71,  72,
         73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,  84,  85,  86,
         87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,  98,  99, 100,
        101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114,
        115, 116, 117, 118, 119, 120, 121, 122, 123,   2])


In [82]:
data = dataset[0]
print(data['source'].shape)


torch.Size([22, 100])
torch.Size([22, 100])
torch.Size([384, 100])


In [32]:
def get_var_seq(var_seq: torch.Tensor, indicate: torch.Tensor):
        # indicate.shape = (batch, max_len + 2)
        result = []
        var_seq = var_seq + len(HaeaVocab.SPECIAL_TOKENS)

        for batch in indicate:
            seq = []
            for item in batch:
                if item in HaeaVocab.SPECIAL_TOKENS:
                    if item == HaeaVocab.SPECIAL_TOKEN_MASK:
                        seq.extend([item for _ in range(len(var_seq))])
                    else:
                        seq.append(item)
                else:
                    seq.extend(var_seq)
                
            result.append(seq)

        result = torch.tensor(result)
        return result


print(get_var_seq(torch.tensor([1, 2, 3]), torch.tensor([[1, 4, 5, 6, 3, 3, 9, 10, 2], [1, 4, 5, 6, 3, 3, 9, 10, 2]])))

tensor([[1, 5, 6, 7, 5, 6, 7, 5, 6, 7, 3, 3, 3, 3, 3, 3, 5, 6, 7, 5, 6, 7, 2],
        [1, 5, 6, 7, 5, 6, 7, 5, 6, 7, 3, 3, 3, 3, 3, 3, 5, 6, 7, 5, 6, 7, 2]])


In [20]:
data = torch.zeros(10, 10)
print(data[0].unsqueeze(0).repeat_interleave(5, dim=0).shape)

torch.Size([5, 10])


In [110]:
import torch
def get_tgt_mask(var_len, max_len, pad_len) -> torch.Tensor:
        matrix = torch.zeros(max_len, max_len)
        matrix[1, 1] = True
        for i in range( (max_len-pad_len-2) // var_len):
            s =  (i * var_len) + 1
            e =  ((i+1) * var_len) + 1
            matrix[s:e, :e] = True
        matrix[-pad_len-1, :-pad_len] = True
      
        return matrix

In [111]:
print(get_tgt_mask(3, 17, 3))

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0.,